In [ ]:
import telebot
import webbrowser
from telebot import types
import requests
from bs4 import BeautifulSoup
import tokenize
from nltk.corpus import stopwords
import pandas as pd
from wordcloud import WordCloud
from PIL import Image
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer
import re
from PIL import Image, ImageOps

bot = telebot.TeleBot('6403375943:AAGw4hg7aXl3KNBCBFDEQyVeSL55eyeKE6o')
bot.set_webhook()

url = 'https://genius.com/artists/Gonefludd/albums'
extra_words = ['припев', 'предприпев', 'интро', 'аутро', 'бридж', 'куплет']
dict_albums = {'DIGITAL FANTAZY' : 2, 'LIL CHILL' : 3, 'VOODOO CHILD' : 4, 'Одиночная Психическая Атака' : 5, 
              'СУПЕРЧУИТС' : 6, 'BOYS DON’T CRY' : 7, 'HIGH LUST' : 8, 'Формы и Пустота' : 9}
list_index = []

tokenizer = RegexpTokenizer(r'\w+')

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
albums = soup.find('ul', class_ = 'ListSectiondesktop__Items-sc-53xokv-8 kbIuNQ').find_all('li')

def get_words(text):
    word_list = tokenizer.tokenize(' '.join(re.split(r"([А-ЯЁ][а-яё]*)", text)).lower())
    filtered_words = [word for word in word_list if word not in stopwords.words('english') 
                          and word not in stopwords.words('russian') and word not in extra_words]
    return dict(pd.Series(filtered_words).value_counts())

@bot.message_handler(commands=['site', 'website'])
def site(message):
    bot.send_message(message.chat.id, f'Открываю..')
    webbrowser.open('https://genius.com/artists/Gonefludd/albums')
    markup = types.ReplyKeyboardMarkup()
    btn1 = types.KeyboardButton('Выбрать альбом')
    btn2 = types.KeyboardButton('Остановить работу бота')
    markup.row(btn1)
    markup.row(btn2)
    bot.send_message(message.chat.id, f'Проверь браузер. Что дальше?', reply_markup=markup)
    bot.register_next_step_handler(message, select_an_album)
    
@bot.message_handler(commands=['start'])
def start(message):
    markup = types.ReplyKeyboardMarkup()
    btn1 = types.KeyboardButton('Выбрать альбом')
    btn2 = types.KeyboardButton('Открыть сайт')
    btn3 = types.KeyboardButton('Остановить работу бота')
    markup.row(btn1)
    markup.row(btn2)
    markup.row(btn3)
    if message.from_user.last_name == None:
        bot.send_message(message.chat.id,
                         f'Привет, {message.from_user.first_name}!\nВыбери действие из списка:',
                         reply_markup=markup)
    else: 
        bot.send_message(message.chat.id, 
                         f'Привет, {message.from_user.first_name} {message.from_user.last_name}!\nВыбери действие из списка:',
                         reply_markup=markup)    
    bot.register_next_step_handler(message, select_an_album)
    
def select_an_album(message):
    if message.text == 'Выбрать альбом':
        markup = types.ReplyKeyboardMarkup()
        btn1 = types.KeyboardButton('DIGITAL FANTAZY')
        btn2 = types.KeyboardButton('LIL CHILL')
        btn3 = types.KeyboardButton('VOODOO CHILD')
        btn4 = types.KeyboardButton('Одиночная Психическая Атака')
        btn5 = types.KeyboardButton('СУПЕРЧУИТС')
        btn6 = types.KeyboardButton('BOYS DON’T CRY')
        btn7 = types.KeyboardButton('HIGH LUST')
        btn8 = types.KeyboardButton('Формы и Пустота')
        markup.row(btn1, btn2, btn3)
        markup.row(btn4)
        markup.row(btn5, btn6)
        markup.row(btn7, btn8)
        bot.send_message(message.chat.id, f'Выбери альбом из предложенного списка:', reply_markup=markup)
        bot.register_next_step_handler(message, save_album)
    elif message.text == 'Открыть сайт' or message.text == '/site':
        bot.send_message(message.chat.id, f'Открываю..')
        webbrowser.open('https://genius.com/artists/Gonefludd/albums')
        markup = types.ReplyKeyboardMarkup()
        btn1 = types.KeyboardButton('Выбрать альбом')
        btn2 = types.KeyboardButton('Остановить работу бота')
        markup.row(btn1)
        markup.row(btn2)
        bot.send_message(message.chat.id, f'Проверь браузер. Что дальше?', reply_markup=markup)
        bot.register_next_step_handler(message, select_an_album)
    elif message.text == '/start':
        markup = types.ReplyKeyboardMarkup()
        btn1 = types.KeyboardButton('Выбрать альбом')
        btn2 = types.KeyboardButton('Открыть сайт')
        btn3 = types.KeyboardButton('Остановить работу бота')
        markup.row(btn1)
        markup.row(btn2)
        markup.row(btn3)
        if message.from_user.last_name == None:
            bot.send_message(message.chat.id, 
                             f'Привет, {message.from_user.first_name}!\nВыбери действие из списка:',
                             reply_markup=markup)
        else: 
            bot.send_message(message.chat.id,
                             f'Привет, {message.from_user.first_name} {message.from_user.last_name}!\nВыбери действие из списка:',
                             reply_markup=markup)    
        bot.register_next_step_handler(message, select_an_album)
    elif message.text == 'Остановить работу бота':
        bot.send_message(message.chat.id, 'Для возобновления нажми /start')
    else:
        bot.send_message(message.chat.id, 'Неккоректный ввод, нажми /start')

def save_album(message):
    try:
        global list_index
        alb_index = dict_albums[message.text]
        list_index.append(alb_index)
        url_tracklist = albums[list_index[0]].find('a')['href']
        response_tracklist = requests.get(url_tracklist)
        soup_tracklist = BeautifulSoup(response_tracklist.text, 'html.parser')
        songs = soup_tracklist.find_all('div', class_ = 'chart_row-content')
        select = ''
        for num, son in enumerate(songs):
            select += str(num) + ' ' + ' '.join(son.text.split()[:-1]) + '\n'
        bot.send_message(message.chat.id, f'Введи номер песни:\n{select.strip()}')
        bot.register_next_step_handler(message, save_song)
    except KeyError:
        markup = types.ReplyKeyboardMarkup()
        btn1 = types.KeyboardButton('Выбрать альбом')
        btn2 = types.KeyboardButton('Остановить работу бота')
        markup.row(btn1)
        markup.row(btn2)
        bot.send_message(message.chat.id, f'Неккоректное название альбома, попробуй ещё раз!', reply_markup=markup)
        bot.register_next_step_handler(message, select_an_album)
    
def save_song(message):
    try:
        global list_index
        markup = types.ReplyKeyboardMarkup()
        btn1 = types.KeyboardButton('Выбрать альбом')
        btn2 = types.KeyboardButton('Остановить работу бота')
        markup.row(btn1)
        markup.row(btn2)
        song_index = message.text
        list_index.append(song_index)
        get_text(list_index[0], int(list_index[1]))
        list_index = []
        file = open('WordCloud.png', 'rb')
        bot.send_message(message.chat.id, f'Облако слов:')
        bot.send_photo(message.chat.id, file, reply_markup=markup)
        bot.register_next_step_handler(message, select_an_album)
    except (IndexError, ValueError):
        list_index.pop()
        list_index.pop()
        markup = types.ReplyKeyboardMarkup()
        btn1 = types.KeyboardButton('Выбрать альбом')
        btn2 = types.KeyboardButton('Остановить работу бота')
        markup.row(btn1)
        markup.row(btn2)
        bot.send_message(message.chat.id, f'Неккоректный номер песни, попробуй ещё раз!', reply_markup=markup)
        bot.register_next_step_handler(message, select_an_album)
        
def get_text(ind_a, ind_s):
    url_tracklist = albums[ind_a].find('a')['href']
    response_tracklist = requests.get(url_tracklist)
    soup_tracklist = BeautifulSoup(response_tracklist.text, 'html.parser')
    songs = soup_tracklist.find_all('div', class_ = 'chart_row-content')
    url_text = songs[ind_s].find('a').get('href')
    response_text = requests.get(url_text)
    soup_text = BeautifulSoup(response_text.text, 'html.parser')
    full_text =''
    for part in soup_text.find_all('div', class_ = 'Lyrics__Container-sc-1ynbvzw-5 Dzxov'):
        full_text += part.text
    split_words = get_words(full_text)
    get_cloud(split_words)
    
def get_cloud(words):
    wordcloud = WordCloud(width = 900,
                        height = 600,
                        background_color = 'black',
                        min_font_size = 5,
                        colormap ='cool')
    wordcloud.generate_from_frequencies(frequencies = words)
    plt.figure(figsize = (10, 7))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.savefig('WordCloud.png')
    
bot.polling(non_stop=True)